In [102]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder
from imblearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

## Load Analytical Base Table

In [104]:
df=pd.read_csv('../data/processed_data.csv',usecols=lambda column:not column.startswith('Unnamed'))
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,614,Spain,Female,40,3,113348.50,1,1,1,77789.01,0
1,758,France,Female,34,1,154139.45,1,1,1,60728.89,0
2,541,Germany,Female,39,9,100116.67,1,1,1,199808.10,1
3,481,France,NotSpecified,37,8,152303.66,2,1,1,175082.20,0
4,711,France,Female,37,8,113899.92,1,0,0,80215.20,0


In [105]:
df.shape

(10000, 11)

In [106]:
x=df.drop('Exited',axis=1)
y=df['Exited']
print(x.shape)
print(y.shape)

(10000, 10)
(10000,)


In [107]:
def perc_target(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    df2=pd.DataFrame(np.array(kv).T,columns=['Exited','Count'])
    df2['Count']=df2['Count'].astype('int64')
    df2['%']=round(df2['Count']/a.shape*100,2)
    return df2.sort_values('Count',ascending=False)

In [108]:
perc_target(y)

,Exited,Count,%
0,0,7963,79.63
1,1,2037,20.37


In [109]:
num_col=x.select_dtypes(include='number').columns.to_list()
num_col

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [110]:
cat_col=x.select_dtypes(include='object').columns.to_list()
cat_col

['Geography', 'Gender']

## Create a Train Test Split

In [112]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(7000, 10) (3000, 10) (7000,) (3000,)


In [113]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 5663 to 1289
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      7000 non-null   int64  
 1   Geography        7000 non-null   object 
 2   Gender           7000 non-null   object 
 3   Age              7000 non-null   int64  
 4   Tenure           7000 non-null   int64  
 5   Balance          7000 non-null   float64
 6   NumOfProducts    7000 non-null   int64  
 7   HasCrCard        7000 non-null   int64  
 8   IsActiveMember   7000 non-null   int64  
 9   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 601.6+ KB


## Pre-processing Pipeline
### Scale numerical data and encode categorical data
###### Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder
###### Create lists of indexes from the list of column names
###### Need to be numeric not string to specify columns name in column transformer

In [115]:
num_feature=[]
for i in num_col:
    loc=x.columns.get_loc(i)
    num_feature.append(loc)

num_feature

[0, 3, 4, 5, 6, 7, 8, 9]

In [116]:
cat_feature=[]
for i in cat_col:
    loc=x.columns.get_loc(i)
    cat_feature.append(loc)

cat_feature

[1, 2]

In [117]:
preprocessor=make_column_transformer(
    (MinMaxScaler(),num_feature),
    (OneHotEncoder(sparse_output=False),cat_feature)
)
preprocessor

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [0, 3, 4, 5, 6, 7, 8, 9]),
                                ('onehotencoder',
                                 OneHotEncoder(sparse_output=False), [1, 2])])

### Build Model Pipeline with SMOTE
##### We are going to use the Pipeline from the imblearn package in place of scikit-learn Pipeline.
##### It takes care automatically to re-sample when called fit() on the pipeline, and does not re-sample test data (when called transform() or predict()).

In [119]:
import xgboost as xgb
model=make_pipeline(
    preprocessor,
    SMOTE(random_state=10),
    xgb.XGBClassifier(use_label_encoder=False,random_state=10,eval_metric='logloss')
)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  [1, 2])])),
                ('smote', SMOTE(random_state=10)),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=10, ...))])

In [120]:
from sklearn.model_selection import GridSearchCV

param_grid = {'xgbclassifier__gamma': [0.5, 0.8, 1],
              'xgbclassifier__max_depth': [3, 4, 5, 6],
              'xgbclassifier__n_estimators': [50, 100, 200]
        }
xgb_grid = GridSearchCV(model, param_grid, verbose=3, cv= 5, n_jobs=4, scoring='f1_macro')

In [121]:
xgb_grid.fit(x_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [0, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8,
                                                                          9]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(sparse_output=False),
                                                                         [1,
                                                                          2])])),
                                       ('smote', SMOTE(random_state=10)),
                                       ('xgbclassifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_...
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=10, ...))]),
             n_jobs=4,
             param_grid={'xgbclassifier__gamma': [0.5, 0.8, 1],
                         'xgbclassifier__max_depth': [3, 4, 5, 6],
                         'xgbclassifier__n_estimators': [50, 100, 200]},
             scoring='f1_macro', verbose=3)

In [122]:
print(xgb_grid.best_params_)

{'xgbclassifier__gamma': 0.5, 'xgbclassifier__max_depth': 4, 'xgbclassifier__n_estimators': 50}


In [123]:
print(xgb_grid.best_score_)

0.7531109553536739


In [124]:
print(f'Trainging Data Score: {xgb_grid.score(x_train,y_train)}')
print(f'Testing Data Score: {xgb_grid.score(x_test,y_test)}')

Trainging Data Score: 0.7947735404457503
Testing Data Score: 0.7619234994071233


In [145]:
prediction=xgb_grid.predict(x_test)
prediction

array([1, 0, 0, ..., 0, 0, 1])

In [149]:
cm=confusion_matrix(y_test,prediction)
print(cm)

[[2119  235]
 [ 245  401]]


In [153]:
cm=np.around(cm/cm.sum(axis=1)[:,np.newaxis],2)
print(cm)

[[0.9  0.1 ]
 [0.38 0.62]]


In [155]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2354
           1       0.63      0.62      0.63       646

    accuracy                           0.84      3000
   macro avg       0.76      0.76      0.76      3000
weighted avg       0.84      0.84      0.84      3000



In [157]:
import joblib
filename='../Models/xgboost.sav'
joblib.dump(xgb_grid,filename)

['../Models/xgboost.sav']

In [159]:
xgb_model=joblib.load(filename)
xgb_model.score(x_test,y_test)

0.7619234994071233